### Important
The taboo_data folder is not included, but is needed for creating the human and original data csv files. For where to download, please check the root README.md

In [1]:
import os
import openai

import pandas as pd
import os
import numpy as np
from ast import literal_eval
import json
import pickle
import time

In [16]:
default_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}"'

In [6]:
openai.api_key =  # removed, paste your own here

In [7]:
def request_response_from_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        #prompt=prompt,
        messages=[
            {"role": "system", "content": "You are a crowdsourcing worker that earns a living through creating paraphrases."},
            {"role": "user", "content": prompt}],
        temperature=1,
        frequency_penalty=0.0,
        presence_penalty=1.5,
        n=2)
    return response

In [8]:
def request_with_checks(prompt):
    success = False
    count = 0
    while not success:
        if count > 0:
            print(f'Retrying with again. Current number of retries: {count}')
        if count >= 5:
            raise Exception('Too many attempts')
        try:
            response = request_response_from_gpt(prompt)
            sucess = True
            break
        except openai.error.RateLimitError as e:
            print(e)
            time.sleep(10)
            count += 1
        except openai.error.APIConnectionError as e:
            print(e)
            time.sleep(5)
            count += 1
        except openai.error.APIError or openai.error.JSONDecodeError:
            time.sleep(5)
            count += 1
    return response

# FB

In [9]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_standard_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [20]:
dct_phrases = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']

In [23]:
dct_phrases.keys()

dict_keys(['get_directions', 'get_distance', 'get_estimated_arrival', 'get_estimated_departure', 'get_estimated_duration', 'get_info_road_condition', 'get_info_route', 'get_info_traffic', 'get_location', 'update_directions'])

In [24]:
dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(default_prompt.format(phrase))

### no taboo words

In [25]:
dct_responses = {}

In [26]:
# collect data

for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(2)

0
Rephrase an original question or statement 3 times. Original phrase: "What is the fastest way to the U2 concert?"
Rephrase an original question or statement 3 times. Original phrase: "Get me the driving directions to Nashville Tn."
Rephrase an original question or statement 3 times. Original phrase: "Find a route with no tolls from here to Tampa"
Rephrase an original question or statement 3 times. Original phrase: "How to get to the jewelry store from the highway"
Rephrase an original question or statement 3 times. Original phrase: "I need to get the directions to Katie 's house in Robertsdale, leaving at 1 : 00 PM."
Rephrase an original question or statement 3 times. Original phrase: "What is the safest route to the San Antonio Airport tomorrow morning"
Rephrase an original question or statement 3 times. Original phrase: "How do I get to the mall"
Rephrase an original question or statement 3 times. Original phrase: "Whats the driving directions to the Walmart near downtown"
Rephrase

Rephrase an original question or statement 3 times. Original phrase: "when will we be in Arkansas if we leave now."
Rephrase an original question or statement 3 times. Original phrase: "if i leave at 8 : 15 will i make my 9 am flight in the morning due to the construction on i - 90"
Rephrase an original question or statement 3 times. Original phrase: "will I be able make it to work on time in this traffic?"
Rephrase an original question or statement 3 times. Original phrase: "What is the ETA to my house?"
Rephrase an original question or statement 3 times. Original phrase: "If I leave here at 8 am, what time will I get to Miami"
Rephrase an original question or statement 3 times. Original phrase: "What time will I get to work if I take I - 90 at 8 am?"
Rephrase an original question or statement 3 times. Original phrase: "can i get to marcus movie theatre by 8 : 15 pm tonight"
Rephrase an original question or statement 3 times. Original phrase: "Can I get from my house to New York City 

Rephrase an original question or statement 3 times. Original phrase: "how long to get to vacaville from here with traffic"
Rephrase an original question or statement 3 times. Original phrase: "How long will it take to get downtown during rush hour"
Rephrase an original question or statement 3 times. Original phrase: "What is current drive time from Tacoma to Seattle"
Rephrase an original question or statement 3 times. Original phrase: "How long will it take me to get to Atlanta with the construction on I - 95?"
Rephrase an original question or statement 3 times. Original phrase: "how long is the drive from detroit to Atlanta"
Rephrase an original question or statement 3 times. Original phrase: "How long is my drive to the grocery store"
Rephrase an original question or statement 3 times. Original phrase: "How long of a drive is it from San Francisco to Monterrey"
Rephrase an original question or statement 3 times. Original phrase: "How long it will take to go to the game tonight"
Rephr

Rephrase an original question or statement 3 times. Original phrase: "what is the traffic like at 5 pm"
Rephrase an original question or statement 3 times. Original phrase: "How 's the traffic flow right now on the 202"
Rephrase an original question or statement 3 times. Original phrase: "show me local car traffic jams"
Rephrase an original question or statement 3 times. Original phrase: "What is the traffic like in Portland at the moment"
Rephrase an original question or statement 3 times. Original phrase: "Can you tell me if traffic is congested between Wilshire and 3rd Street in Los Angeles?"
Rephrase an original question or statement 3 times. Original phrase: "are there any accidents on i - 26 west between here and chapin"
Rephrase an original question or statement 3 times. Original phrase: "roads with the least traffic"
Rephrase an original question or statement 3 times. Original phrase: "How is traffic on I - 10"
Rephrase an original question or statement 3 times. Original phrase

In [28]:
with open('challenge_data/fb/fb_0_v1.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [2]:
file = open("challenge_data/fb/fb_0_v1.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [3]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])

In [6]:
fb_0 = pd.DataFrame.from_dict(dct_df)
fb_0 = fb_0.sample(frac=0.5, replace=False, random_state=1)

In [7]:
fb_0.to_csv('challenge_data/fb/fb_0.csv', index=False)

### ADD TWO TABOO WORDS

In [16]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_restricted_2_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [42]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}” or “{}” in your responses.'

In [43]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1]))

In [45]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "What is the fastest way to the U2 concert?". Don’t use the words “directions” or “direction” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Get me the driving directions to Nashville Tn.". Don’t use the words “directions” or “direction” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Find a route with no tolls from here to Tampa". Don’t use the words “directions” or “direction” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How to get to the jewelry store from the highway". Don’t use the words “directions” or “direction” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "I need to get the directions to Katie 's house in Robertsdale, leaving at 1 : 00 PM.". Don’t use the words “directions” or “direction” in your responses.
Rephrase an original question or 

Rephrase an original question or statement 3 times. Original phrase: "How many miles is it from here to Disney World". Don’t use the words “far” or “distance” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "how far is it to drive between houston and vancouver". Don’t use the words “far” or “distance” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Mileage, by car, between Alabama and Georgia". Don’t use the words “far” or “distance” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "how far is it to el paso". Don’t use the words “far” or “distance” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What is the distance from Florida to California". Don’t use the words “far” or “distance” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How far away in miles is New York from Los Angeles". Don’

Rephrase an original question or statement 3 times. Original phrase: "I need to be at my meeting in Waco at 3 pm so will I make it if I leavy by 1 : 30?". Don’t use the words “eta” or “arrival” in your responses.
3
Rephrase an original question or statement 3 times. Original phrase: "What time should I set off to the football game that 's starting at 5?". Don’t use the words “leave” or “need” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What time do I need to leave my house to make it to the 8 pm movie?". Don’t use the words “leave” or “need” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What time should i leave to make it to the Jazz game at 8 PM?". Don’t use the words “leave” or “need” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What time should I leave the office to make it to the Dr 's office by 5 pm?". Don’t use the words “leave” or “need” in your res

Rephrase an original question or statement 3 times. Original phrase: "how long is the drive from detroit to Atlanta". Don’t use the words “long” or “time” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long is my drive to the grocery store". Don’t use the words “long” or “time” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long of a drive is it from San Francisco to Monterrey". Don’t use the words “long” or “time” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long it will take to go to the game tonight". Don’t use the words “long” or “time” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Get the time to Walmart using Jabaquara Street.". Don’t use the words “long” or “time” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "how long will it take me to get to Shreveport if

Rephrase an original question or statement 3 times. Original phrase: "Is there snow on 243 going down?". Don’t use the words “icy” or “flooding” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Are the roads to Ellen 's house clear?". Don’t use the words “icy” or “flooding” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Is there ice on my street?". Don’t use the words “icy” or “flooding” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Are there any reports of black ice this morning on the road to the campground?". Don’t use the words “icy” or “flooding” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Are the road icy going to Shelbys house?". Don’t use the words “icy” or “flooding” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What are the road conditions for my commute to work this

Rephrase an original question or statement 3 times. Original phrase: "Water parks in Minneapolis". Don’t use the words “station” or “where” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "is there any grocery store nearby me". Don’t use the words “station” or “where” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Where is Iolani Palace?". Don’t use the words “station” or “where” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Is Brian 's house close to the H1 Freeway?". Don’t use the words “station” or “where” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the nearest movie theatre?". Don’t use the words “station” or “where” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "where is the closest six flags theme park". Don’t use the words “station” or “where” in your responses.


In [46]:
with open('challenge_data/fb/fb_2_v1.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [11]:
file = open("challenge_data/fb/fb_2_v1.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [13]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
            

In [ ]:
fb_2 = pd.DataFrame.from_dict(dct_df)
fb_2.to_csv('challenge_data/fb/fb_2.csv', index=False)

In [17]:
# WE CAN FILTER FOR TABOO WORDS
fb_2 = pd.read_csv('challenge_data/fb/fb_2.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_2['no_taboo_words'] = fb_2.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_2 = fb_2[fb_2['no_taboo_words'] == True]

fb_2.to_csv('challenge_data/fb/fb_2_filt.csv', index=False)

### ADD FOUR TABOO WORDS

In [20]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_restricted_4_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [51]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}”, “{}”, “{}” or “{}” in your responses.'

In [52]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2], dct_taboo[key][3]))

In [54]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "What is the fastest way to the U2 concert?". Don’t use the words “directions”, “direction”, “avoid” or “quickest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Get me the driving directions to Nashville Tn.". Don’t use the words “directions”, “direction”, “avoid” or “quickest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Find a route with no tolls from here to Tampa". Don’t use the words “directions”, “direction”, “avoid” or “quickest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How to get to the jewelry store from the highway". Don’t use the words “directions”, “direction”, “avoid” or “quickest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "I need to get the directions to Katie 's house in Robertsdale, leaving at 1 : 00 PM.". Don’t use the wo

Rephrase an original question or statement 3 times. Original phrase: "How far is Chicago from here". Don’t use the words “far”, “distance”, “miles” or “mileage” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How far is the Wine and Food Festival in Orlando?". Don’t use the words “far”, “distance”, “miles” or “mileage” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Distance between Louisville and Lexington". Don’t use the words “far”, “distance”, “miles” or “mileage” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What is the distance between San Diego and San Francisco". Don’t use the words “far”, “distance”, “miles” or “mileage” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How many miles is it from here to Disney World". Don’t use the words “far”, “distance”, “miles” or “mileage” in your responses.
Rephrase an original

Rephrase an original question or statement 3 times. Original phrase: "If I leave my house at 8 am will I make it to Billys house by 8 : 30 am". Don’t use the words “eta”, “arrival”, “late” or “possible” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Will I be delayed leaving at 8 am to get to Kearney by 11". Don’t use the words “eta”, “arrival”, “late” or “possible” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "If I leave from home at 8 : 00 am, what time would I arrive in Myrtle Beach, South Carolina??". Don’t use the words “eta”, “arrival”, “late” or “possible” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long until I arrive". Don’t use the words “eta”, “arrival”, “late” or “possible” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "will i get to the HNL airport in 15 minutes when i leave before 5 pm via Nimitz". 

Rephrase an original question or statement 3 times. Original phrase: "how early do i have to leave to get to work". Don’t use the words “leave”, “need”, “should” or “want” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "I need to be at OMSI in Portland by 2 : 30, what time should I leave?". Don’t use the words “leave”, “need”, “should” or “want” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "My 5 o'clock appointment in Gardena has been pushed to 6 pm. Can you give me a new departure time?". Don’t use the words “leave”, “need”, “should” or “want” in your responses.
4
Rephrase an original question or statement 3 times. Original phrase: "what is the flight time between Orlando & Louisville". Don’t use the words “long”, “time”, “length” or “longer” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "how long is my drive from the shoe store to my home". Don’t use the words

Rephrase an original question or statement 3 times. Original phrase: "is roads still bad this morning from the snow storm?". Don’t use the words “icy”, “flooding”, “flooded” or “flood” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Did Black Bear wash flood last night?". Don’t use the words “icy”, “flooding”, “flooded” or “flood” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Is Interstate 57 clear of snow now?". Don’t use the words “icy”, “flooding”, “flooded” or “flood” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "will the weather affect my commute". Don’t use the words “icy”, “flooding”, “flooded” or “flood” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Will the roads be flooded this evening?". Don’t use the words “icy”, “flooding”, “flooded” or “flood” in your responses.
Rephrase an original question or statement 

Rephrase an original question or statement 3 times. Original phrase: "What is the traffic like in Portland at the moment". Don’t use the words “accidents”, “delays”, “traffic” or “closures” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Can you tell me if traffic is congested between Wilshire and 3rd Street in Los Angeles?". Don’t use the words “accidents”, “delays”, “traffic” or “closures” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "are there any accidents on i - 26 west between here and chapin". Don’t use the words “accidents”, “delays”, “traffic” or “closures” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "roads with the least traffic". Don’t use the words “accidents”, “delays”, “traffic” or “closures” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How is traffic on I - 10". Don’t use the words “accidents”, “delays

Rephrase an original question or statement 3 times. Original phrase: "are there any shorter routes home". Don’t use the words “alternate”, “reroute”, “alternative” or “other” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "find alternate route please". Don’t use the words “alternate”, “reroute”, “alternative” or “other” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "find an alternate route to work". Don’t use the words “alternate”, “reroute”, “alternative” or “other” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Redirect route". Don’t use the words “alternate”, “reroute”, “alternative” or “other” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Can you give me alternate routes to avoid traffic". Don’t use the words “alternate”, “reroute”, “alternative” or “other” in your responses.
Rephrase an original question or statemen

In [55]:
with open('challenge_data/fb/fb_4_v1.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [21]:
file = open("challenge_data/fb/fb_4_v1.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [22]:
# prepare data

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_4 = pd.DataFrame.from_dict(dct_df)
fb_4.to_csv('challenge_data/fb/fb_4.csv', index=False)

In [24]:
# WE CAN FILTER FOR TABOO WORDS
fb_4 = pd.read_csv('challenge_data/fb/fb_4.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_4['no_taboo_words'] = fb_4.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_4 = fb_4[fb_4['no_taboo_words'] == True]

fb_4.to_csv('challenge_data/fb/fb_4_filt.csv', index=False)

### ADD SIX FINAL TABOO WORDS

In [27]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_restricted_6_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [62]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}”, “{}”, “{}”, “{}”, “{}” or “{}” in your responses.'

In [63]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2], dct_taboo[key][3], dct_taboo[key][4], dct_taboo[key][5]))

In [65]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "What is the fastest way to the U2 concert?". Don’t use the words “directions”, “direction”, “avoid”, “quickest”, “without” or “least” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Get me the driving directions to Nashville Tn.". Don’t use the words “directions”, “direction”, “avoid”, “quickest”, “without” or “least” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Find a route with no tolls from here to Tampa". Don’t use the words “directions”, “direction”, “avoid”, “quickest”, “without” or “least” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How to get to the jewelry store from the highway". Don’t use the words “directions”, “direction”, “avoid”, “quickest”, “without” or “least” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "I need to get the direc

Rephrase an original question or statement 3 times. Original phrase: "What is the distance between Seattle and Chicago". Don’t use the words “far”, “distance”, “miles”, “mileage”, “kilometers” or “between” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How far is Phx from NY". Don’t use the words “far”, “distance”, “miles”, “mileage”, “kilometers” or “between” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How far between Portland and Seattle". Don’t use the words “far”, “distance”, “miles”, “mileage”, “kilometers” or “between” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "WHAT IS THE DISTANCE FROM SEATTLE TO REYKJAVIK". Don’t use the words “far”, “distance”, “miles”, “mileage”, “kilometers” or “between” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How far is Chicago from here". Don’t use the words “far”, “distance”, 

Rephrase an original question or statement 3 times. Original phrase: "when will mom get to our house from blytheville if she leaves now". Don’t use the words “eta”, “arrival”, “late”, “possible”, “arriving” or “if” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "If I leave right now, can I get downtown by 7 : 30?". Don’t use the words “eta”, “arrival”, “late”, “possible”, “arriving” or “if” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Can i get to Detroit by 12 if I leave now?". Don’t use the words “eta”, “arrival”, “late”, “possible”, “arriving” or “if” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "When will I reach my destination". Don’t use the words “eta”, “arrival”, “late”, “possible”, “arriving” or “if” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Can I get from here to Dallas before midnight if I leave now?". 

Rephrase an original question or statement 3 times. Original phrase: "What time should I leave to make it to the Dodger game by 5 pm?". Don’t use the words “leave”, “need”, “should”, “want”, “have” or “start” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "When to I need to leave to get airport by 9 : 00 am?". Don’t use the words “leave”, “need”, “should”, “want”, “have” or “start” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What time should I leave to make it to work by 9 am?". Don’t use the words “leave”, “need”, “should”, “want”, “have” or “start” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "What time do I need to leave my house to get to Grand Blvd Movies by 7 pm?". Don’t use the words “leave”, “need”, “should”, “want”, “have” or “start” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "I need to be at my nephew 's 

Rephrase an original question or statement 3 times. Original phrase: "Tell me how long it takes to get to Orlando if I take 40". Don’t use the words “long”, “time”, “length”, “longer”, “fast” or “hours” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long would it take to drive from the Brooklyn Bridge to New Jersey". Don’t use the words “long”, “time”, “length”, “longer”, “fast” or “hours” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long does it take to get from Orlando to Destin?". Don’t use the words “long”, “time”, “length”, “longer”, “fast” or “hours” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long will it take to drive to Lake Wales". Don’t use the words “long”, “time”, “length”, “longer”, “fast” or “hours” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "How long will it take to get to my 2 pm appo

6
Rephrase an original question or statement 3 times. Original phrase: "is there a carpool lane ahead". Don’t use the words “tolls”, “gas”, “route”, “way”, “on” or “there” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "will this route take me by a gas station". Don’t use the words “tolls”, “gas”, “route”, “way”, “on” or “there” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Where are gas stations along my route". Don’t use the words “tolls”, “gas”, “route”, “way”, “on” or “there” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Find an ATM along my route to Stephanie 's office". Don’t use the words “tolls”, “gas”, “route”, “way”, “on” or “there” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Tell me the Shell stations along I - 59 between here and Hattiesburg.". Don’t use the words “tolls”, “gas”, “route”, “way”, “on” or “

Rephrase an original question or statement 3 times. Original phrase: "What is the traffic like in Downtown Phoenix". Don’t use the words “accidents”, “delays”, “traffic”, “closures”, “wrecks” or “detours” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "is this traffic jam almost over". Don’t use the words “accidents”, “delays”, “traffic”, “closures”, “wrecks” or “detours” in your responses.
8
Rephrase an original question or statement 3 times. Original phrase: "Where is the closest Target". Don’t use the words “station”, “where”, “stop”, “closest”, “nearest” or “there” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "is there a bus stop near my new office". Don’t use the words “station”, “where”, “stop”, “closest”, “nearest” or “there” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Water parks in Minneapolis". Don’t use the words “station”, “where”, “stop”, “closes

Rephrase an original question or statement 3 times. Original phrase: "Any way around this traffic". Don’t use the words “alternate”, “reroute”, “alternative”, “other”, “another” or “shorter” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Are there any short cuts". Don’t use the words “alternate”, “reroute”, “alternative”, “other”, “another” or “shorter” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Is there a faster route to get to Phoenix Ave". Don’t use the words “alternate”, “reroute”, “alternative”, “other”, “another” or “shorter” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Is there a route that has lighter traffic than 1st ave". Don’t use the words “alternate”, “reroute”, “alternative”, “other”, “another” or “shorter” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "Can you find me a faster route". Don’t use the w

In [66]:
with open('challenge_data/fb/fb_6_v1.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [26]:
file = open("challenge_data/fb/fb_6_v1.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [28]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_6 = pd.DataFrame.from_dict(dct_df)
fb_6.to_csv('challenge_data/fb/fb_6.csv', index=False)

In [30]:
# WE CAN FILTER FOR TABOO WORDS
fb_6 = pd.read_csv('challenge_data/fb/fb_6.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_6['no_taboo_words'] = fb_6.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_6 = fb_6[fb_6['no_taboo_words'] == True]

fb_6.to_csv('challenge_data/fb/fb_6_filt.csv', index=False)

#### finalize data into one csv

In [32]:
# filtered data without taboo samples

fb_0 = pd.read_csv('challenge_data/fb/fb_0.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_2 = pd.read_csv('challenge_data/fb/fb_2.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_4 = pd.read_csv('challenge_data/fb/fb_4.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_6 = pd.read_csv('challenge_data/fb/fb_6.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

In [37]:
# data with taboo samples

fb_0 = pd.read_csv('challenge_data/fb/fb_0.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_2 = pd.read_csv('challenge_data/fb/fb_2_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_4 = pd.read_csv('challenge_data/fb/fb_4_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_6 = pd.read_csv('challenge_data/fb/fb_6_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

Proceed with either taboo samples or without them

In [39]:
fb = fb_0.append(fb_2).append(fb_4).append(fb_6).drop_duplicates('text')

In [40]:
dct = {'get_directions': 0,
 'get_distance': 1,
 'get_estimated_arrival': 2,
 'get_estimated_departure': 3,
 'get_estimated_duration': 4,
 'get_info_road_condition': 5,
 'get_info_route': 6,
 'get_info_traffic': 7,
 'get_location': 8,
 'update_directions': 9}

def function(x):
    return dct[x]

fb['label'] = fb['intent'].apply(function)
fb = fb.sample(frac=1).reset_index(drop=True)
fb['text'] = fb['text'].apply(lambda x: x.replace('"',''))

In [41]:
fb[['text', 'label']].to_csv('challenge_data/fb/full_fb_gpt_filt.csv', index=False) # replace name for taboo and non taboo

Now lets prepare user data and the original data into dataset - for human data we have the same preprocessing and for the  original we remove the seed samples, take only needed intents and prepare csv

To get the data for the original version and human version from the original paper, please see the README.md file at the root

In [36]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_standard_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [37]:
dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

In [38]:
fb_h_0 = pd.DataFrame.from_dict(dct_dataframe)

In [39]:
f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_restricted_2_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_2 = pd.DataFrame.from_dict(dct_dataframe)

f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_restricted_4_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_4 = pd.DataFrame.from_dict(dct_dataframe)


f = open('/taboo_data/challenge_data/intent_classification/facebook/facebook_restricted_6_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_6 = pd.DataFrame.from_dict(dct_dataframe)

In [40]:
fb_h = fb_h_0.append(fb_h_2).append(fb_h_4).append(fb_h_6).drop_duplicates('text')

In [41]:
fb_h['label'] = fb_h['intent'].apply(function)
fb_h = fb_h.sample(frac=1).reset_index(drop=True)
fb_h['text'] = fb_h['text'].apply(lambda x: x.replace('"',''))

In [42]:
fb_h[['text', 'label']].to_csv('full_fb_human.csv', index=False)

In [167]:
intents = list(set(fb_h['intent']))

#### NOW ORIGINAL WITHOUT SEEDS

In [6]:

f = open('../../Larson_iterative/taboo_data/challenge_data/intent_classification/facebook/train.json')

# returns JSON object as 
# a dictionary
data = json.load(f)


# Closing file
f.close()

In [3]:

f = open('/taboo_data/challenge_data/intent_classification/facebook/test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)


# Closing file
f.close()

In [7]:
all_seed_phrases = []
for key in dct_phrases:
    all_seed_phrases.extend(dct_phrases[key])

NameError: name 'dct_phrases' is not defined

In [8]:
fin_dct = {}
for key in data.keys():
    if key not in all_seed_phrases:
        fin_dct[key] = data[key]

In [9]:
dct_dataframe = {'text': [], 'intent': []}

for key in data:
    dct_dataframe['intent'].append(data[key])
    dct_dataframe['text'].append(key)

tmp_orig = pd.DataFrame.from_dict(dct_dataframe)

In [195]:
tmp_orig = tmp_orig[tmp_orig['intent'].isin(intents)]

In [196]:
tmp_orig = tmp_orig.sample(frac=1).reset_index(drop=True)
tmp_orig['label'] = tmp_orig['intent'].apply(function)

In [197]:
tmp_orig[['text', 'label']].to_csv('full_fb_orig_test.csv', index=False)

## LETS TRAIN SVM 

In [3]:
full_gpt = pd.read_csv('challenge_data/fb/full_fb_gpt_filt.csv').dropna().reset_index(drop=True)
full_human = pd.read_csv('challenge_data/fb/full_fb_human.csv').dropna().reset_index(drop=True)
full_orig = pd.read_csv('challenge_data/fb/full_fb_orig_train.csv').dropna().reset_index(drop=True)

full_orig_test = pd.read_csv('challenge_data/fb/full_fb_orig_test.csv').dropna().reset_index(drop=True)

In [4]:
import re
import string

full_gpt['text']=full_gpt['text'].apply(lambda x: x.lower())
full_human['text']=full_human['text'].apply(lambda x: x.lower())
full_orig['text']=full_orig['text'].apply(lambda x: x.lower())
full_orig_test['text']=full_orig_test['text'].apply(lambda x: x.lower())

full_gpt['text']=full_gpt['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_human['text']=full_human['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_orig['text']=full_orig['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_orig_test['text']=full_orig_test['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

full_gpt['text']=full_gpt['text'].apply(lambda x: re.sub(' +',' ',x))
full_human['text']=full_human['text'].apply(lambda x: re.sub(' +',' ',x))
full_orig['text']=full_orig['text'].apply(lambda x: re.sub(' +',' ',x))
full_orig_test['text']=full_orig_test['text'].apply(lambda x: re.sub(' +',' ',x))

In [9]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from joblib import dump, load
from sklearn import tree
import numpy as np
import itertools
from sklearn import svm
from sklearn.model_selection import train_test_split

gpt_res = []
gpt_f1 = []

human_res = []
human_f1 = []

orig_res = []
orig_f1 = []

for idx in range(0, 10):

    train_gpt, test_gpt = train_test_split(full_gpt, test_size=0.2)
    train_human, test_human = train_test_split(full_human, test_size=0.2)
    
    tmp_train = full_orig.sample(frac=0.2, replace=False, random_state=1)
    tmp_test = full_orig_test.sample(frac=1, replace=False, random_state=1)
    
    pipeline = Pipeline([
            ('bow', CountVectorizer()),  
            ('tfidf', TfidfTransformer()),  
            ('c', svm.SVC(probability=True))
        ])
    fit = pipeline.fit(train_human['text'].str.lower(), train_human['label'])
    
    pred=pipeline.predict(test_gpt['text'].str.lower())
    gpt_res.append(accuracy_score(pred,test_gpt['label']))
    gpt_f1.append(f1_score(pred,test_gpt['label'],  average = 'weighted'))
    
    pred=pipeline.predict(test_human['text'].str.lower())
    human_res.append(accuracy_score(pred,test_human['label']))
    human_f1.append(f1_score(pred,test_human['label'],  average = 'weighted'))
    
    pred=pipeline.predict(tmp_test['text'].str.lower())
    orig_res.append(accuracy_score(pred,tmp_test['label']))
    orig_f1.append(f1_score(pred,tmp_test['label'],  average = 'weighted'))

In [10]:
arr = np.array(gpt_res)

print("ACC GPT MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(gpt_f1)

print("F1 GPT MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr))) 

arr = np.array(human_res)

print("ACC HUMAN SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(human_f1)

print("F1 HUMAN SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(orig_res)

print("ACC ORIG SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(orig_f1)

print("F1 ORIG SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

ACC GPT MEAN: 0.8803030303030303 STD: 0.008866797361793738
F1 GPT MEAN: 0.8804690906552862 STD: 0.008634912939361625
ACC HUMAN SAME MEAN: 0.9452342487883685 STD: 0.006480184529929417
F1 HUMAN SAME MEAN: 0.9452948148505425 STD: 0.0065063237014759
ACC ORIG SAME MEAN: 0.6658458813108946 STD: 0.00941855684409622
F1 ORIG SAME MEAN: 0.6453919122213808 STD: 0.009566952620823837
